In [ ]:
#   Uncertain what is needed here.   Code was written with AI assistance, and I have eliminated packages that were unused.
import pandas as pd
import xarray as xr
import holoviews as hv
import os
import ipywidgets as ipw
import panel as pn
import panel.widgets as pnw
from holoviews import opts
from holoviews.operation.datashader import datashade
hv.extension('bokeh')

import matplotlib.pyplot as plt         
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import cv2



### check sample photograph (optional)
you can skip this step

In [ ]:
from IPython.display import Image, display

def display_image_with_grid_and_scale(image_path, grid_size=10, scale_length_cm=2, scale_bar_position_cm=(0, 0), figsize_multiplier=0.015):
    # Load the image
    image = cv2.imread(image_path)

    # Convert image to RGB (OpenCV uses BGR by default)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Get image dimensions
    height, width, _ = image_rgb.shape

    # Create a grid overlay
    for x in range(0, width, grid_size):
        cv2.line(image_rgb, (x, 0), (x, height), color=(255, 0, 0), thickness=2)

    for y in range(0, height, grid_size):
        cv2.line(image_rgb, (0, y), (width, y), color=(255, 0, 0), thickness=2)

    # Calculate proportional figsize
    figsize = (width * figsize_multiplier, height * figsize_multiplier)

    # Display the image with grid and scale bar
    plt.figure(figsize=figsize)
    plt.imshow(image_rgb)
    # plt.axis('on')  # Turn off axis labels
    plt.xticks(fontsize=35)                                                     # Set font size for X and Y axis labels
    plt.yticks(fontsize=35)
    # plt.yticks(fontsize=40)

    plt.show()

# Example: Display photo of sample for reference (Optional)

image_path = r"C:\path\file.jpg"
display_image_with_grid_and_scale(image_path, 
                                  grid_size=205,                                # grid size is self explanatory

)



### load X, Y, Z data

In [3]:
# Step 1: Load the Data

file_path = r'C:\scripting\Profilometer\Lot 710A(10)\710A(10)bottom-1_Height  - Thresholded (302.8 µm ; 474.9 µm).txt'  #This is your data file you want to plot.  It should be 3 columns, X, Y , Z

df = pd.read_csv(file_path, sep='\t', names=['x', 'y', 'z'], dtype={'x': float, 'y': float, 'z': 'str'})
df['z'] = pd.to_numeric(df['z'], errors='coerce')
df = df.dropna()                                    # Option 1 - Drop rows with NaN values or empty sets

# Extract the file name from the path and take the first 50 characters
file_name = os.path.basename(file_path)
title = file_name[:50]

# Convert the dataframe to an Xarray Dataset
ds = xr.Dataset.from_dataframe(df.dropna().set_index(['x', 'y']))

### Generate heat map, make adjustments to Z

In [ ]:
# Set the desired z-value limits (color limits)
z_min = -30  # Adjust this value as needed
z_max = 120   # Adjust this value as needed

# Create the heatmap with specified z-value limits and colorbar label
heatmap = hv.HeatMap(df, kdims=['x', 'y'], vdims='z')
heatmap.opts(cmap='seismic', colorbar=True, title=title, xlabel='X (mm)', ylabel='Y (mm)', 
             width=800, height=600, clim=(z_min, z_max),
             colorbar_opts={'title': 'Height (μm)'})  # Add units to the colorbar label

# Display the heatmap.  For some reason when I render this, the download doesn't work so it has to be done manually.
heatmap



In [6]:
# this is very time consuming.   
v.save(heatmap, title+'.png', fmt='png')


###  This may take time to render.  Be patient.  The sliders are nice

In [ ]:
def plot_height_by_x(x):
    y_values = ds.sel(x=x)['y'].values
    z_values = ds.sel(x=x)['z'].values
    curve = hv.Curve((y_values, z_values), 'Y (mm)', 'Height (um)')
    return curve.opts(title=f'{title}__Height Profile at X = {x}', width=800, height=300)

def plot_height_by_y(y):
    x_values = ds.sel(y=y)['x'].values
    z_values = ds.sel(y=y)['z'].values
    curve = hv.Curve((x_values, z_values), 'X (mm)', 'Height (um)')
    return curve.opts(title=f'{title}__Height Profile at Y = {y}', width=800, height=300)

x_slider = pn.widgets.FloatSlider(start=float(ds['x'].min()), end=float(ds['x'].max()), step=0.1, name='X (mm)')
y_slider = pn.widgets.FloatSlider(start=float(ds['y'].min()), end=float(ds['y'].max()), step=0.1, name='Y (mm)')

x_stream = hv.streams.Stream.define('X', x=x_slider.value)()
y_stream = hv.streams.Stream.define('Y', y=y_slider.value)()

x_dmap = hv.DynamicMap(plot_height_by_x, streams=[x_stream])
y_dmap = hv.DynamicMap(plot_height_by_y, streams=[y_stream])

x_slider.param.watch(lambda event: x_stream.event(x=event.new), 'value')
y_slider.param.watch(lambda event: y_stream.event(y=event.new), 'value')

layout = pn.Column(
    heatmap,
    pn.Row(x_slider, y_slider),
    x_dmap,
    y_dmap
)
layout.servable()